### 0) Description

This notebook contains code to clean pulled from the following https://www.sports-reference.com/ websites:
    - College Football: https://www.sports-reference.com/cfb/
    - NFL: https://www.pro-football-reference.com/
    

### 1) Import packages

In [1]:
import pandas as pd

### 2) Clean data

In [13]:
# clean nfl team stats data

# read data
df = pd.read_csv('nfl_team_stats_1999_2019.csv')

# set column order
df_clean=df[df.columns[[13,10,11,1,9,12,6,4,5,2,8,7,3,0]]]

# turn off warnings
pd.options.mode.chained_assignment = None  # default='warn'

# add Playoff column to indicate if team made it to playoffs
df_clean['Playoff'] = df_clean['Tm'].str.contains('\*|\+')
# remove '*' and '+' characters from team name
df_clean['Tm'] = df_clean['Tm'].replace('\*|\+','', regex=True)

# fill nans with 0 in Ties column
df_clean['T'] = df_clean['T'].fillna(0).astype(int)

# save as csv
df_clean.to_csv('nfl_team_stats_1999_2019_clean.csv', index=False)

In [2]:
# clean nfl draft data

# read data
df = pd.read_csv('nfl_draft_2000_2020.csv')

# drop columns that are not relevant for our project
cols_to_drop = ['To','AP1','PB','St','DrAV','G','Cmp','Att','Yds','TD','Int','Att.1','Yds.1','TD.1','Rec','Yds.2','TD.2','Solo','Int.1','Sk','Unnamed: 28_level_1']
df.drop(cols_to_drop, inplace=True, axis=1)

# save as csv
df.to_csv('nfl_draft_2000_2020_clean_with_AV.csv', index=False)

In [15]:
# clean nfl combine data

# read data
df = pd.read_csv('nfl_combine_2000_2020.csv')

# convert feet & inches to inches
df[['Feet','Inches']] = df['Ht'].str.split('-',expand=True).astype(int)
df['Ht'] = df['Feet']*12 + df['Inches']

# split and clean Drafted column: e.g New York Jets / 1st / 13th pick / 2000
df[['Team','Round','Pick','Yr']] = df['Drafted (tm/rnd/yr)'].str.split(' / ',expand=True)

# remove non-numeric characters
df['Round'] = df['Round'].str.replace(r'\D+', '')
df['Pick'] = df['Pick'].str.replace(r'\D+', '')

# fill nans with 999
df['Round'] = df['Round'].fillna(999).astype(int)
df['Pick'] = df['Pick'].fillna(999).astype(int)

# drop unnecessary columns
cols_to_drop = ['Yr','Feet','Inches','Drafted (tm/rnd/yr)','College']
df.drop(cols_to_drop, inplace=True, axis=1)

# save as csv
df.to_csv('nfl_combine_2000_2020_clean.csv', index=False)

In [16]:
# clean team offense and defense data

# read data
df_off = pd.read_csv('ncaaf_team_offense_2000_2019.csv')
df_def = pd.read_csv('ncaaf_team_defense_2000_2019.csv')

# rename columns
df_off.columns = ['Year','Rk','School','G','Pts','Pass_Cmp','Pass_Att','Pass_Pct','Pass_Yds','Pass_TD','Rush_Att','Rush_Yds','Rush_Avg','Rush_TD','Off_Plays','Off_Yds','Off_Avg','FD_Pass','FD_Rush','FD_Pen','FD_Tot','Pen','Pen_Yds','Fum','Int','TO_Tot']
df_def.columns = ['Year','Rk','School','G','Opp_Pts','Opp_Pass_Cmp','Opp_Pass_Att','Opp_Pass_Pct','Opp_Pass_Yds','Opp_Pass_TD','Opp_Rush_Att','Opp_Rush_Yds','Opp_Rush_Avg','Opp_Rush_TD','Opp_Off_Plays','Opp_Off_Yds','Opp_Off_Avg','Opp_FD_Pass','Opp_FD_Rush','Opp_FD_Pen','Opp_FD_Tot','Opp_Pen','Opp_Pen_Yds','Opp_Fum','Opp_Int','Opp_TO_Tot']

# join dataframes
key_cols = ['Year','School','G']
df = pd.merge(df_off, df_def,  how='inner', left_on=key_cols, right_on = key_cols)
df.rename(columns={'Rk_x':'Off_Rk','Rk_y':'Def_Rk'}, inplace=True)

# save as csv
df.to_csv('ncaaf_team_offense_defense_2000_2019_clean.csv', index=False)

In [2]:
# clean player data

# read data
df = pd.read_csv('ncaa_all_player_ind_stats_extended_v2.csv')

# fill nans with 0 in columns
list_cols = ['Def_Solo_Tackles','Def_Ast_Tackles','Def_Tot_Tackles','Def_Loss_Tackles','Def_Sk','Def_Int','Def_Int_Yds','Def_Int_Avg','Def_Int_TD','Def_Int_PD','Def_FR','Def_Yds_Fumbles','Def_TD_Fumbles','Def_FF','Kick_Ret','Kick_ret_Yds','Kick_ret_Avg','Kick_ret_TD','Punt_Ret','Punt_ret_Yds','Punt_ret_Avg','Punt_ret_TD','Rush_Att','Rush_Yds','Rush_Avg','Rush_TD','Rec','Rec_Yds','Rec_Avg','Rec_TD','Scrim_Plays','Scrim_Yds','Scrim_Avg','Scrim_TD','Punts','Punt_Yds','Punt_Avg','Kick_XPM','Kick_XPA','Kick_XP_pct','Kick_FGM','Kick_FGA','Kick_FG_pct','Kick_Pts','Pass_Cmp','Pass_Att','Pass_Pct','Pass_Yds','Pass_Y/A','Pass_AY/A','Pass_TD','Pass_Int','Pass_Rate']
df[list_cols] = df[list_cols].fillna(0)

# remove '*' characters from year
df['Year'] = df['Year'].str.replace(r'\*', '')

# save as csv
df.to_csv('ncaa_all_player_ind_stats_extended_clean_v2.csv', index=False) 

In [2]:
# clean team ratings data

# read data
df = pd.read_csv('ncaaf_team_ratings_1999_2019.csv')

# rename columns
df.columns = ['Year','Rk','School','Conf','AP Rank','W','L','OSRS','DSRS','SRS','PPG','Opp_PPG','Pass_YPA','Opp_Pass_YPA','Rush_YPA','Opp_Rush_YPA','Total_YPP','Opp_Total_YPP']

# save as csv
df.to_csv('ncaaf_team_ratings_1999_2019_clean.csv', index=False) 